# Explicit chaining 

Explicit chaining is when an existing exception is deliberately associated with a new exception object at the point when the exception object is raised.  This is done in the process of translating an exception from one type to another.  Consider the example below:

In [1]:
import math

def inclination(dx, dy):
    return math.degrees(math.atan(dy / dx))

The above `inclination` function returns the slope in degrees given the horizontal and vertical distance components of a line.  This works for most slopes:

In [2]:
inclination(3, 5)

59.03624346792648

However it fails with `ZeroDivisionError` when the horizontal component, `dx` is zero:

In [3]:
inclination(0, 5)

ZeroDivisionError: division by zero

Now modify the code by introducing a new exception type, `InclinationError`, and an exception handler for the `ZeroDivisionError` that wallows the active exception and raises a new one - translating one exception to another:

In [5]:
class InclinationError(Exception):
    pass

def inclination(dx, dy):
    try:
        return math.degrees(math.atan(dy / dx))
    except ZeroDivisionError as e:
        raise InclinationError("Slope cannot be vertical") from e

The syntax for _explicit_ exception chaining here has been included with the `from e` suffix when the exception is created.  This associates the new exception object with the original exception `e`. However, unlike _implicit_ chaining which associates the chained exception through the `__context__` attribute, explicit chaining associates the chained exception through the `__cause__` attribute.  See what happens when the exception is triggered:

In [6]:
inclination(0, 5)

InclinationError: Slope cannot be vertical

Similar to implicitly chained exceptions, the default stack trace report includes the chained exception.  To examine the chained exception directly you can use the `__cause__` attribute of the most recent exception:

In [7]:
try:
    inclination(0,5)
except InclinationError as e:
    print(e)
    print(e.__cause__)

Slope cannot be vertical
division by zero


When translating exceptions in this way - typically at module boundaries where implementation details should be hidden by raising exceptions that clients can reasonably expect - consider whether to explicitly chain root cause exceptions to improve diagnostics and aid debugging.